In [1]:
import sys
import os
import numpy as np
import pandas as pd
import torch
from PIL import Image

import ipywidgets as widgets
from io import BytesIO

from ganpreffinder import GANPrefFinder
from jupyterGUI import GUIses
from jupyterComponentsExprGUI import GUICompExprSes

StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.


### Description:

Possible datasets: ffhq, car, cat, church, horse, bedrooms, kitchen, places

---

In [2]:
DEVICE = "cpu"
if torch.cuda.is_available():
    DEVICE = "cuda:0"

print("Device used:", DEVICE)

Device used: cpu


In [3]:
gpf = GANPrefFinder(class_name="car",
                    gan_sample_seed=0,
                    n_comp_in_use=10,
                    adaptive_components=[0,1,2,3,4,5,6,7,8,9],
                    strength_left_bound=-10,
                    strength_right_bound=10,
                    ppbo_max_iter_fMAP_estimation=5000,
                    device=DEVICE)

In [4]:
gui_comp_ses = GUICompExprSes(gpf)
gui_comp_ses.run()

In [5]:
guises = GUIses(gpf)
guises.run()

In [ ]:
# CONTINUE

---